In [ ]:
import s4bi
from importlib import reload
from scipy import constants
from astropy.cosmology import Planck15
import qubic
from qubic import mcmc
import healpy as hp
rc('figure',figsize=(16,4))
rc('font',size=12)

import fgbuster


# CMB-S4 Description

In [ ]:
rc('figure',figsize=(16,4))
rc('font',size=12)
subplot(1,2,1)
errorbar(s4bi.s4_config['frequency'], s4bi.s4_config['depth_p'], xerr=s4bi.s4_config['bandwidth']/2, fmt='ro')
xlabel('Frequency [GHz]')
ylabel(r'Depth_p [$\mu$K.arcmin]')
title('CMB-S4 Configuration')
subplot(1,2,2)
errorbar(s4bi.s4_config['frequency'], s4bi.s4_config['fwhm'], xerr=s4bi.s4_config['bandwidth']/2, fmt='ro')
xlabel('Frequency [GHz]')
ylabel('FWHM [arcmin]')
title('CMB-S4 Configuration')

In [ ]:
print(s4bi.s4_config['frequency'], s4bi.s4_config['depth_p'])

# QUBIC+ Description

In [ ]:
rc('figure',figsize=(16,4))
rc('font',size=12)
reload(s4bi)
# QUBIC+
qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])

qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
print(qp_config['frequency'])
print(qp_config['depth_p'])
print(qp_config['bandwidth'])

subplot(1,2,1)
errorbar(s4bi.s4_config['frequency'], s4bi.s4_config['depth_p'], xerr=s4bi.s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
errorbar(qp_config['frequency'], qp_config['depth_p'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
xlabel('Frequency [GHz]')
ylabel(r'Depth_p [$\mu$K.arcmin]')
title('S4-BI Configuration')
legend()
subplot(1,2,2)
errorbar(s4bi.s4_config['frequency'], s4bi.s4_config['fwhm'], xerr=s4bi.s4_config['bandwidth']/2, fmt='ro', label='CMB-S4')
errorbar(qp_config['frequency'], qp_config['fwhm'], xerr=qp_config['bandwidth']/2, fmt='bo', label='BI')
xlabel('Frequency [GHz]')
ylabel('FWHM [arcmin]')
title('S4-BI Configuration')
legend()

figure()
subplot(1,2,1)
plot([150., 220], [1.4, 1.2], 'ro', label='Measured on MC [Mousset et al. 2021]')
mynus = np.linspace(75, 300,100)
plot(mynus, s4bi.fct_subopt(mynus),label='Extrapolation')
xlabel('Frequency [GHz]')
ylabel('Suboptimality on r')
title('BI Suboptimality')
legend()

# Component maps

In [ ]:
reload(s4bi)
components = ['c1', 'd0', 's0']
ref_freqs = [150., 353., 70.]
fsky = 0.03
nside = 256
radec_center = [0, -57]
center = qubic.equ2gal(radec_center[0], radec_center[1])

map_CMB, map_dust_353GHz, map_sync_70GHz = s4bi.get_component_maps(components, ref_freqs, nside, fsky)


In [ ]:
stokes = ['I', 'Q', 'U', 'P']
okpix = map_CMB[0,:] != hp.UNSEEN

m_sync_70GHz = np.mean(map_sync_70GHz[:,okpix], axis=1)
s_sync_70GHz = np.std(map_sync_70GHz[:,okpix], axis=1)

m_dust_353GHz = np.mean(map_dust_353GHz[:,okpix], axis=1)
s_dust_353GHz = np.std(map_dust_353GHz[:,okpix], axis=1)

m_CMB = np.mean(map_CMB[:,okpix], axis=1)
s_CMB = np.std(map_CMB[:,okpix], axis=1)


for istk in range(4):
    hp.gnomview(map_sync_70GHz[istk,:], rot=center, reso=15, 
                title='Synchrotron '+stokes[istk]+' at 70 GHz ($\mu$K$_{CMB}$)'+'\n {0:5.2g}+/-{1:5.2g}'.format(m_sync_70GHz[istk], s_sync_70GHz[istk]), sub=(1,4,istk+1))

figure()    
for istk in range(4):
    hp.gnomview(map_dust_353GHz[istk,:], rot=center, reso=15, 
                title='Dust '+stokes[istk]+' at 353 GHz ($\mu$K$_{CMB}$)'+'\n {0:5.2g}+/-{1:5.2g}'.format(m_dust_353GHz[istk], s_dust_353GHz[istk]), sub=(1,4,istk+1))

figure()    
for istk in range(4):
    hp.gnomview(map_CMB[istk,:], rot=center, reso=15, 
                title='CMB '+stokes[istk]+' ($\mu$K$_{CMB}$)'+'\n {0:5.2g}+/-{1:5.2g}'.format(m_CMB[istk], s_CMB[istk]), sub=(1,4,istk+1))



# Component Models

FGBuster gives components scaled to 1 at their respective reference frequencies. We have added a double-beta component model.

In [ ]:
rc('figure',figsize=(16,8))
rc('font',size=12)

import fgbuster
from fgbuster import CMB, Dust, Synchrotron, AnalyticComponent, ModifiedBlackBody
reload(fgbuster)

dust_ref_freq = 353.
sync_ref_freq = 70.
dust = Dust(dust_ref_freq, beta_d=1.54, temp=20)
cmb = CMB() 
sync = Synchrotron(sync_ref_freq, beta_pl=-3)

### The double beta component model
dbdust = fgbuster.component_model.Dust_2b(nu0=353, temp=20)
print(dbdust.params)   # parameters are beta0, beta1 and nubreak
# let's fix the params for now
av_beta = 1.54
dbeta = -0.5
beta0 = av_beta+dbeta
beta1 = av_beta-dbeta
nubreak = 100.

numin = 20
numax = 400
nnu = 1000
nus = np.linspace(numin, numax, nnu)

plot(nus, sync.eval(nus), label=r'Synchrotron($\nu_r$={} GHz)'.format(70))
plot(nus, cmb.eval(nus), label='CMB')
plot(nus, dust.eval(nus), label=r'Dust ($\nu_r$={} GHz)'.format(353))
plot(nus, dbdust.eval(nus, beta0, beta1, nubreak), 
     label=r'$2\beta$ Dust with $\beta_0$={0:4.2f}, $\beta_1$={1:4.2f}, $\nu_0$={2:3.0f} GHz'.format(beta0, beta1, nubreak))
axvline(x=nubreak,ls=':', color='k',label=r'Break at $\nu_0$={0:3.0f} GHz'.format(nubreak))
xlabel('Frequency [GHz]')
ylabel('SED')
yscale('log')
xscale('log')
legend()
title('All normalized to 1 at a ref frequency')

print(dust.eval(150)/dust.eval(353))

In [ ]:
### Play with width
widths = np.array([0.1, 0.25, 0.5, 0.75, 1.])
mynus = np.linspace(80, 120, 100)
for i in range(len(widths)):
    dbdust = fgbuster.component_model.Dust_2b(nu0=353, temp=20, break_width=widths[i])
    plot(mynus, dbdust.eval(mynus, beta0, beta1, nubreak), label='Width = {}'.format(widths[i]))
xlabel('Frequency [GHz]')
ylabel('SED')
# yscale('log')
# xscale('log')
legend()


Now we use the normalization (RMS) found from PYSM in our region ito scale them:

In [ ]:
mystk = [0,3]
beta0 = 1.39
beta1 = 1.69
nubreak = 100.

ii=0
for istk in mystk:
    subplot(1,2,ii+1)
    ii+=1
    plot(nus, sync.eval(nus) * s_sync_70GHz[istk], label=r'Synchrotron($\nu_r$={} GHz)'.format(70))
    plot(nus, cmb.eval(nus) * s_CMB[istk], label='CMB')
    plot(nus, dust.eval(nus) * s_dust_353GHz[istk], label=r'Dust ($\nu_r$={} GHz)'.format(353))
    plot(nus, dbdust.eval(nus, beta0, beta1, nubreak) * s_dust_353GHz[istk], 
         label=r'$2\beta$ Dust with $\beta_0$={}, $\beta_1$={}, $\nu_0$={} GHz'.format(beta0, beta1, nubreak))
    axvline(x=nubreak,ls=':', color='k')
    xlabel('Frequency [GHz]')
    ylabel(r'SED [$\mu K_{CMB}$]')
    yscale('log')
    xscale('log')
    title('Stokes '+stokes[istk]+' in clean QUBIC region {}%'.format(fsky*100))
    legend()
tight_layout()

# Dust Models

In [ ]:
dust_ref_freq = 353.
sync_ref_freq = 70.
dust_model = Dust(dust_ref_freq, temp=20)
sync_model = Synchrotron(sync_ref_freq)
double_dust_model = fgbuster.component_model.Dust_2b(nu0=353, temp=20)


### Classes to have the band integration

In [ ]:
rc('figure',figsize=(16,12))
rc('font',size=12)

class foreground_model:
    def __init__(self, dust_type, nus, dnus, dust_model, sync_model, double_dust_model, integrate=True, ninteg=11):
        self.dust_type = dust_type
        self.nus = nus
        self.dnus = dnus
        self.dust_model = dust_model
        self.sync_model = sync_model
        self.double_dust_model = double_dust_model

        self.integrate = integrate
        self.ninteg = ninteg
        if integrate:
            self.nus_bands = np.zeros((len(nus), ninteg))
            for i in range(len(self.nus)):
                numin = self.nus[i] - self.dnus[i]/2
                numax = self.nus[i] + self.dnus[i]/2
                self.nus_bands[i,:] = np.linspace(numin, numax, self.ninteg)
        
            
    def SED_mono(self, x, pars):
        if self.dust_type == '1beta':
            sync_amp = pars[0]
            sync_pl = pars[1]
            cmb = pars[2]
            dust_amp = pars[3]
            beta_dust = pars[4]
            dust = dust_amp * self.dust_model.eval(x, beta_dust) 
            sync = sync_amp * self.sync_model.eval(x, sync_pl)
        else:
            sync_amp = pars[0]
            sync_pl = pars[1]
            cmb = pars[2]
            dust_amp = pars[3]
            beta0_dust = pars[4]
            beta1_dust = pars[5]
            nubreak_dust = pars[6]
            dust = dust_amp * self.double_dust_model.eval(x, beta0_dust, beta1_dust, nubreak_dust) 
            sync = sync_amp * self.sync_model.eval(x, sync_pl)            
        return cmb + dust + sync

    def __call__(self, x, pars, extra_args=None):
        if self.integrate:
            values = np.mean(self.SED_mono(self.nus_bands, pars), axis=1)
        else:
            values = self.SED_mono(self.nus, pars)
        return values
    
                
nubreak =110

pars_1beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], 1.54])                
pars_2beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], 1.49, 1.59, nubreak])                


integrate = True
ninteg = 11
fg_S4_1b = foreground_model('1beta', s4bi.s4_config['frequency'], 
                            s4bi.s4_config['bandwidth'], 
                            dust_model, sync_model, double_dust_model,
                            integrate=integrate, ninteg=ninteg)
fg_S4_2b = foreground_model('2beta', s4bi.s4_config['frequency'], 
                            s4bi.s4_config['bandwidth'], 
                            dust_model, sync_model, double_dust_model,
                            integrate=integrate, ninteg=ninteg)

fg_qp_1b = foreground_model('1beta', qp_config['frequency'], 
                            qp_config['bandwidth'], 
                            dust_model, sync_model, double_dust_model,
                            integrate=integrate, ninteg=ninteg)
fg_qp_2b = foreground_model('2beta', qp_config['frequency'], 
                            qp_config['bandwidth'], 
                            dust_model, sync_model, double_dust_model,
                            integrate=integrate, ninteg=ninteg)


#### Test optimization
# %timeit fg_S4_1b.SED_mono(s4bi.s4_config['frequency'], pars_1beta)
# %timeit fg_S4_1b(s4bi.s4_config['frequency'], pars_1beta)
# %timeit fg_qp_1b.SED_mono(qp_config['frequency'], pars_1beta)
# %timeit fg_qp_1b(qp_config['frequency'], pars_1beta)




subplot(3,2,1)
title('CMB-S4')
plot(nus, fg_S4_1b.SED_mono(nus, pars_1beta), 'g', label='Dirac 1beta')
plot(nus, fg_S4_2b.SED_mono(nus, pars_2beta), 'm', label='Dirac 2beta')
errorbar(s4bi.s4_config['frequency'], 
         fg_S4_1b(s4bi.s4_config['frequency'], pars_1beta), 
         xerr = s4bi.s4_config['bandwidth']/2,
         fmt='ro', label='Band Integrated 1beta')
errorbar(s4bi.s4_config['frequency'], 
         fg_S4_2b(s4bi.s4_config['frequency'], pars_2beta), 
         xerr = s4bi.s4_config['bandwidth']/2,
         fmt='bo', label='Band Integrated 2beta')
axvline(x=nubreak, ls=':', color='k')
yscale('log')
legend()

subplot(3,2,2)
title('QP')
plot(nus, fg_qp_1b.SED_mono(nus, pars_1beta), 'g', label='Dirac 1beta')
plot(nus, fg_qp_2b.SED_mono(nus, pars_2beta), 'm', label='Dirac 2beta')
errorbar(qp_config['frequency'], 
         fg_qp_1b(qp_config['frequency'], pars_1beta), 
         xerr = qp_config['bandwidth']/2,
         fmt='go', label='Band Integrated 1beta')
errorbar(qp_config['frequency'], 
         fg_qp_2b(qp_config['frequency'], pars_2beta), 
         xerr = qp_config['bandwidth']/2,
         fmt='mo', label='Band Integrated 2beta')
axvline(x=nubreak, ls=':', color='k')
yscale('log')
legend()


subplot(3,2,3)
title('CMB-S4: Ratio between Mono and Integrate')
errorbar(s4bi.s4_config['frequency'], 
         fg_S4_1b(s4bi.s4_config['frequency'], pars_1beta) / fg_S4_1b.SED_mono(s4bi.s4_config['frequency'], pars_1beta), 
         xerr = s4bi.s4_config['bandwidth']/2,
         fmt='ro', label='Band Integrated 1beta')
errorbar(s4bi.s4_config['frequency'], 
         fg_S4_2b(s4bi.s4_config['frequency'], pars_2beta) / fg_S4_2b.SED_mono(s4bi.s4_config['frequency'], pars_2beta), 
         xerr = s4bi.s4_config['bandwidth']/2,
         fmt='bo', label='Band Integrated 2beta')
axvline(x=nubreak, ls=':', color='k')
#yscale('log')
legend()

subplot(3,2,4)
title('QP: Ratio between Mono and Integrate')
errorbar(qp_config['frequency'], 
         fg_qp_1b(qp_config['frequency'], pars_1beta) / fg_qp_1b.SED_mono(qp_config['frequency'], pars_1beta), 
         xerr = qp_config['bandwidth']/2,
         fmt='go', label='Band Integrated 1beta')
errorbar(qp_config['frequency'], 
         fg_qp_2b(qp_config['frequency'], pars_2beta) / fg_qp_2b.SED_mono(qp_config['frequency'], pars_2beta), 
         xerr = qp_config['bandwidth']/2,
         fmt='mo', label='Band Integrated 2beta')
axvline(x=nubreak, ls=':', color='k')
#yscale('log')
legend()



subplot(3,1,3)
title('Ratio to 1 beta')
plot(nus, fg_S4_2b.SED_mono(nus, pars_2beta) / fg_S4_1b.SED_mono(nus, pars_1beta), 'b', label='Dirac 2beta')
errorbar(s4bi.s4_config['frequency'], 
         fg_S4_2b(s4bi.s4_config['frequency'], pars_2beta) / fg_S4_1b(s4bi.s4_config['frequency'], pars_1beta), 
         xerr = s4bi.s4_config['bandwidth']/2,
         fmt='ro', label='CMB-S4')
errorbar(qp_config['frequency'], 
         fg_qp_2b(qp_config['frequency'], pars_2beta) / fg_qp_1b(qp_config['frequency'], pars_1beta), 
         xerr = qp_config['bandwidth']/2,
         fmt='bo', label='QP')
axvline(x=nubreak, ls=':', color='k')
# yscale('log')
legend()

tight_layout()

# First test: fitting the single beta with a true single beta (Integrated or not)

In [ ]:
reload(s4bi)
reload(mcmc)

integrate=True
ninteg = 11

pars_true = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], 1.54])
#vals = model_1beta(nus, pars_true)

deg2 = 1240
arcmins_squared = deg2*60**2   #1240 = 3% of the sky
myfsky = 1240/(4*np.pi*(180/np.pi)**2)


#### S4 Data
nus_s4 = s4bi.s4_config['frequency']
bw_s4 = s4bi.s4_config['bandwidth']
class_fg_S4_1b = foreground_model('1beta', nus_s4, bw_s4, 
                            dust_model, sync_model, double_dust_model, 
                            integrate=integrate, ninteg=ninteg)
def fg_S4_1b(x, pars, extra_args=None):
    return class_fg_S4_1b(x, pars)

sed_s4 = fg_S4_1b(nus_s4, pars_true)
err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared)


#### S4BI Data
qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])
qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
nus_qp = qp_config['frequency']
bw_qp = qp_config['bandwidth']
class_fg_qp_1b = foreground_model('1beta', nus_qp, bw_qp, 
                            dust_model, sync_model, double_dust_model, 
                            integrate=integrate, ninteg=ninteg)
def fg_qp_1b(x, pars, extra_args=None):
    return class_fg_qp_1b(x, pars)

sed_qp = fg_qp_1b(nus_qp, pars_true)
err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared)


rc('figure',figsize=(8,8))
rc('font',size=12)

figure()
#plot(nus, vals)
errorbar(nus_qp, sed_qp, yerr=err_sed_qp, fmt='bo', capsize=4, label='S4-BI')
errorbar(nus_s4, sed_s4, yerr=err_sed_s4, fmt='ro', capsize=4, label='CMB-S4')
yscale('log')
xscale('log')
xlabel('Frequency [GHz]')
ylabel(r'SED [$\mu K_{CMB}$]')
legend()


from qubic import mcmc
reload(mcmc)
p0 = pars_true.copy()

fixedpars = np.array([0, 0, 0, 0, 0])



fp = np.array([p0, np.abs(p0)*0+0.01]).T
print(np.shape(fp))

ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
        p0=p0, model=fg_S4_1b, fixedpars=fixedpars)
nbmc = 3000
samp = ll.run(nbmc)
flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)


ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
        p0=p0, model=fg_qp_1b, fixedpars=fixedpars)
nbmc = 3000
samp_qp = ll_qp.run(nbmc)
flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)


from getdist import plots, MCSamples

allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta'])
alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta_{dust}'])

names = allnames[fixedpars==0]
labels = alllabels[fixedpars==0]
mytrue = pars_true[fixedpars==0]

mkrs = {}
for i in range(len(names)):
    mkrs[names[i]]=mytrue[i]

samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, samps_qp], filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, samps_qp], ['CMB', 'DustAmp', 'DustBeta'], filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)
suptitle('Marginalized over Synchrotron')
tight_layout()

averages_s4 = np.mean(flat_samples_s4, axis=0)
errors_s4 = np.std(flat_samples_s4, axis=0)
averages_qp = np.mean(flat_samples_qp, axis=0)
errors_qp = np.std(flat_samples_qp, axis=0)
print('Average_S4',averages_s4)
print('Sigma_S4',errors_s4)
print('Average_qp',averages_qp)
print('Sigma_qp',errors_qp)
print()
for i in range(len(names)):
    print('{0:20s}: CMBS4: {1:7.3f} +/- {2:7.3f}      -     S4BI: {3:7.3f} +/- {4:7.3f}'.format(names[i], averages_s4[i], errors_s4[i], averages_qp[i], errors_qp[i]))

### Conversion of err_CMB into 
thecmb = where(names == 'CMB')[0][0]
Delta_r_CMB_s4 = averages_s4[thecmb]**2/0.345
Delta_r_CMB_qp = averages_qp[thecmb]**2/0.345
print('Delta r CMB S4 = {0:6.5g}'.format(Delta_r_CMB_s4))
print('Delta r CMB QP = {0:6.5g}'.format(Delta_r_CMB_qp))

### Chi2
mypars_s4 = np.zeros(5)
mypars_s4[fixedpars==1] = p0[fixedpars==1]
mypars_s4[fixedpars==0] = averages_s4
mypars_qp = np.zeros(5)
mypars_qp[fixedpars==1] = p0[fixedpars==1]
mypars_qp[fixedpars==0] = averages_qp
# print(mypars_s4)
# print(mypars_qp)

chi2_s4 = np.sum(((sed_s4 - fg_S4_1b(nus_s4, mypars_s4))/err_sed_s4)**2)
ndf_s4 = len(sed_s4)-len(averages_s4)
chi2_qp = np.sum(((sed_qp - fg_qp_1b(nus_qp, mypars_qp))/err_sed_qp)**2)
ndf_qp = len(sed_qp)-len(averages_qp)
print('CMBS4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_s4, ndf_s4, chi2_s4/ndf_s4))
print('BI-S4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_qp, ndf_qp, chi2_qp/ndf_qp))



# Second: Fitting a double-beta with the single beta

In [ ]:
rc('figure',figsize=(8,8))
rc('font',size=12)
reload(s4bi)
reload(mcmc)
from getdist import plots, MCSamples


def doit(nubreak, av_beta, dbeta, integrate=True, ninteg=11, doplot=True, verbose=True, subopt=True):
    
    #### True Parameters
    pars_true_1beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], av_beta])
    beta0 = av_beta-dbeta
    beta1 = av_beta
    pars_true_2beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], av_beta-dbeta, av_beta, nubreak])

    #### Survey parameters
    deg2 = 1240
    arcmins_squared = deg2*60**2   #1240 = 3% of the sky
    myfsky = 1240/(4*np.pi*(180/np.pi)**2)
    
    #### S4 Data
    nus_s4 = s4bi.s4_config['frequency']
    bw_s4 = s4bi.s4_config['bandwidth']
    fg_S4_1b = foreground_model('1beta', nus_s4, bw_s4, 
                                dust_model, sync_model, double_dust_model, 
                                integrate=integrate, ninteg=ninteg)
    fg_S4_2b = foreground_model('2beta', nus_s4, bw_s4, 
                                dust_model, sync_model, double_dust_model, 
                                integrate=integrate, ninteg=ninteg)
    sed_s4 = fg_S4_2b(nus_s4, pars_true_2beta)
    err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared)


    #### S4BI Data
    qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
    qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
    qp_suboptimality = np.array([False, False, False, subopt, subopt, subopt, subopt, subopt, subopt])
    qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
    nus_qp = qp_config['frequency']
    bw_qp = qp_config['bandwidth']
    fg_qp_1b = foreground_model('1beta', nus_qp, bw_qp, 
                                dust_model, sync_model, double_dust_model, 
                                integrate=integrate, ninteg=ninteg)
    fg_qp_2b = foreground_model('2beta', nus_qp, bw_qp, 
                                dust_model, sync_model, double_dust_model, 
                                integrate=integrate, ninteg=ninteg)
    sed_qp = fg_qp_2b(nus_qp, pars_true_2beta)
    err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared)

    #### Plot data
    if doplot:
        figure()
        plot(nus, fg_qp_2b.SED_mono(nus, pars_true_2beta), 
             label='2 Beta: {0:5.2f} for nu < {1:} ; {2:5.2f} above'.format(beta0, nubreak, beta1))
        errorbar(nus_qp, sed_qp, yerr=err_sed_qp, fmt='bo', capsize=4, label='S4-BI')
        errorbar(nus_s4, sed_s4, yerr=err_sed_s4, fmt='ro', capsize=4, label='CMB-S4')
        yscale('log')
        xscale('log')
        xlabel('Frequency [GHz]')
        ylabel(r'SED [$\mu K_{CMB}$]')
        axvline(x=nubreak, ls=':', color='k')
        legend()

    #### MCMC runs
    p0 = pars_true_1beta.copy()
    fixedpars = np.array([1, 1, 0, 0, 0])
    
    ## S4
    ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
            p0=p0, model=fg_S4_1b, fixedpars=fixedpars)
    nbmc = 3000
    samp = ll.run(nbmc)
    flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)
    
    ## QP
    ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
            p0=p0, model=fg_qp_1b, fixedpars=fixedpars)
    nbmc = 3000
    samp_qp = ll_qp.run(nbmc)
    flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)

    #### Prepare MCMC Triangle Plot and results
    allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta'])
    alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta_{dust}'])
    names = allnames[fixedpars==0]
    labels = alllabels[fixedpars==0]
    mytrue = pars_true_1beta[fixedpars==0]
    mkrs = {}
    for i in range(len(names)):
        mkrs[names[i]]=mytrue[i]

    samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
    samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)

    #### Triangle Plots
    if doplot:
#         figure()
#         g = plots.getSubplotPlotter()
#         g.settings.scaling=False
#         g.settings.axes_fontsize=12
#         g.triangle_plot([samps, samps_qp], filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)

        figure()
        g = plots.getSubplotPlotter()
        g.settings.scaling=False
        g.settings.axes_fontsize=12
        g.triangle_plot([samps, samps_qp], ['CMB', 'DustAmp', 'DustBeta'], filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)
        suptitle('Marginalized over Synchrotron')
        tight_layout()

    #### Results
    averages_s4 = np.mean(flat_samples_s4, axis=0)
    errors_s4 = np.std(flat_samples_s4, axis=0)
    averages_qp = np.mean(flat_samples_qp, axis=0)
    errors_qp = np.std(flat_samples_qp, axis=0)
    if verbose:
        print()
        for i in range(len(names)):
            print('{0:20s}: CMBS4: {1:7.3f} +/- {2:7.3f}      -     S4BI: {3:7.3f} +/- {4:7.3f}'.format(names[i], averages_s4[i], errors_s4[i], averages_qp[i], errors_qp[i]))
        print()

    ### Conversion of err_CMB into 
    thecmb = where(names == 'CMB')[0][0]
    Delta_r_CMB_s4 = averages_s4[thecmb]**2/0.345
    Delta_r_CMB_qp = averages_qp[thecmb]**2/0.345
    Err_Delta_r_CMB_s4 = errors_s4[thecmb]**2/0.345
    Err_Delta_r_CMB_qp = errors_qp[thecmb]**2/0.345
    
    if verbose:
        print('Delta r CMB S4 = {0:6.5g} +/- {1:6.5g}'.format(Delta_r_CMB_s4, Err_Delta_r_CMB_s4))
        print('Delta r CMB QP = {0:6.5g} +/- {1:6.5g}'.format(Delta_r_CMB_qp, Err_Delta_r_CMB_qp))
        print()

    ### Chi2
    mypars_s4 = np.zeros(5)
    mypars_s4[fixedpars==1] = p0[fixedpars==1]
    mypars_s4[fixedpars==0] = averages_s4
    mypars_qp = np.zeros(5)
    mypars_qp[fixedpars==1] = p0[fixedpars==1]
    mypars_qp[fixedpars==0] = averages_qp

    chi2_s4 = np.sum(((sed_s4 - fg_S4_1b(nus_s4, mypars_s4))/err_sed_s4)**2)
    ndf_s4 = len(sed_s4)-len(averages_s4)
    chi2_qp = np.sum(((sed_qp - fg_qp_1b(nus_qp, mypars_qp))/err_sed_qp)**2)
    ndf_qp = len(sed_qp)-len(averages_qp)
    if verbose:
        print('CMBS4: chi2={0:6.2f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_s4, ndf_s4, chi2_s4/ndf_s4))
        print('BI-S4: chi2={0:6.2f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_qp, ndf_qp, chi2_qp/ndf_qp))

    return names, [samps, averages_s4, errors_s4, Delta_r_CMB_s4, Err_Delta_r_CMB_s4, chi2_s4, ndf_s4], [samps_qp, averages_qp, errors_qp, Delta_r_CMB_qp, Err_Delta_r_CMB_s4, chi2_qp, ndf_qp]
    

#### Run
nubreak = 150
av_beta = 1.54
dbeta = 0.1 
suboptimality=True
integrate=True
ninteg = 11
names, resS4, resQP = doit(nubreak, av_beta, dbeta, integrate=integrate, 
                           ninteg=ninteg, doplot=True, verbose=True, subopt=suboptimality)

print()
dr_s4 = resS4[3]
err_dr_s4 = resS4[4]
chi2_s4 = resS4[5]
ndf_s4 = resS4[6]
print(dr_s4, err_dr_s4, chi2_s4, ndf_s4)

dr_qp = resQP[3]
err_dr_qp = resQP[4]
chi2_qp = resQP[5]
ndf_qp = resQP[6]
print(dr_qp, err_dr_qp, chi2_qp, ndf_qp)



In [ ]:
suboptimality = True
integrate=True
ninteg = 11
av_beta = 1.54
dbeta = 0.1 

numin = 80
numax = 250
nn = 36
all_nubreak = np.linspace(numin, numax, nn)
print(all_nubreak)

dr_qp = np.zeros(nn)
err_dr_qp = np.zeros(nn)
chi2_qp = np.zeros(nn)
ndf_qp = np.zeros(nn)
dr_s4 = np.zeros(nn)
err_dr_s4 = np.zeros(nn)
chi2_s4 = np.zeros(nn)
ndf_s4 = np.zeros(nn)

for i in range(nn):
    print()
    print('{} / {}: nubreak={}'.format(i,nn, all_nubreak[i]))
    names, resS4, resQP = doit(all_nubreak[i], av_beta, dbeta, 
                               integrate=integrate, ninteg=ninteg, 
                               doplot=False, verbose=False, subopt=suboptimality)
    dr_s4[i] = resS4[3]
    err_dr_s4[i] = resS4[4]
    chi2_s4[i] = resS4[5]
    ndf_s4[i] = resS4[6]
    dr_qp[i] = resQP[3]
    err_dr_qp[i] = resQP[4]
    chi2_qp[i] = resQP[5]
    ndf_qp[i] = resQP[6]
    print('CMB-S4 : Delta_r = {0:5.3g} +/- {1:5.3g} - chi2={2:6.2f} ndf={3:3.0f} chi2/ndf={4:5.2f}'.format(dr_s4[i], err_dr_s4[i], chi2_s4[i], ndf_s4[i], chi2_s4[i]/ndf_s4[i]))
    print('QP     : Delta_r = {0:5.3g} +/- {1:5.3g} - chi2={2:6.2f} ndf={3:3.0f} chi2/ndf={4:5.2f}'.format(dr_qp[i], err_dr_qp[i], chi2_qp[i], ndf_qp[i], chi2_qp[i]/ndf_qp[i]))




In [ ]:
subplot(3,1,1)
errorbar(all_nubreak, dr_s4, yerr=err_dr_s4, fmt='ro-', label='S4')
errorbar(all_nubreak, dr_qp, yerr=err_dr_qp, fmt='bo-', label='QP')
axhline(y=0, ls=':', color='k')
axhline(y=5e-4, ls=':', color='g', label='CMB-S4 Science Target')
xlabel(r'$\nu_{break}$ [GHz]')
ylabel('$\Delta r$')
legend(loc='upper left')

subplot(3,1,2)
plot(all_nubreak, chi2_s4, 'ro-', label='S4')
plot(all_nubreak, chi2_qp, 'bo-', label='QP')
xlabel(r'$\nu_{break}$ [GHz]')
ylabel('$\chi^2$')
legend(loc='upper left')

subplot(3,1,3)
plot(all_nubreak, chi2_s4/ndf_s4, 'ro-', label='S4')
plot(all_nubreak, chi2_qp/ndf_s4, 'bo-', label='QP')
xlabel(r'$\nu_{break}$ [GHz]')
ylabel('$\chi^2 / ndf$')
legend(loc='upper left')



# Now: fitting the double beta with a doublebeta

In [ ]:
reload(s4bi)
reload(mcmc)

integrate=True
ninteg = 11

av_beta = 1.54
pars_true_1beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], av_beta])
dbeta = 0.1
nubreak = 250
pars_true_2beta = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], av_beta-dbeta, av_beta+dbeta, nubreak])

deg2 = 1240
arcmins_squared = deg2*60**2   #1240 = 3% of the sky
myfsky = 1240/(4*np.pi*(180/np.pi)**2)


#### S4 Data
nus_s4 = s4bi.s4_config['frequency']
bw_s4 = s4bi.s4_config['bandwidth']
fg_S4_2b = foreground_model('2beta', nus_s4, bw_s4, 
                            dust_model, sync_model, double_dust_model, 
                            integrate=integrate, ninteg=ninteg)
sed_s4 = fg_S4_2b(nus_s4, pars_true_2beta)
err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared)


#### S4BI Data
qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])
qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
nus_qp = qp_config['frequency']
bw_qp = qp_config['bandwidth']
fg_qp_2b = foreground_model('2beta', nus_qp, bw_qp, 
                            dust_model, sync_model, double_dust_model, 
                            integrate=integrate, ninteg=ninteg)
sed_qp = fg_qp_2b(nus_qp, pars_true_2beta)
err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared)


rc('figure',figsize=(8,8))
rc('font',size=12)

figure()
plot(nus, fg_qp_2b.SED_mono(nus, pars_true_2beta))
errorbar(nus_qp, sed_qp, yerr=err_sed_qp, fmt='bo', capsize=4, label='S4-BI')
errorbar(nus_s4, sed_s4, yerr=err_sed_s4, fmt='ro', capsize=4, label='CMB-S4')
yscale('log')
xscale('log')
xlabel('Frequency [GHz]')
ylabel(r'SED [$\mu K_{CMB}$]')
axvline(x=nubreak, ls=':', color='k')
legend()


from qubic import mcmc
reload(mcmc)
p0 = pars_true_2beta.copy()

fixedpars = np.array([0, 0, 0, 0, 0, 0, 0])
#flatpriors = np.array([[0.2, 0.35], [-3.1, -2.9], [-0.1, 0.1], [13.,19.], [1., 2.], [1., 2.], [70, 250]])
flatpriors = None

ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
        p0=p0, model=fg_S4_2b, fixedpars=fixedpars)
nbmc = 3000
samp = ll.run(nbmc)
flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)


ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
        p0=p0, model=fg_qp_2b, fixedpars=fixedpars)
nbmc = 3000
samp_qp = ll_qp.run(nbmc)
flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)


from getdist import plots, MCSamples

allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'])
alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta^0_{dust}', r'\beta^1_{dust}', r'\nu^{break}_{dust}'])

names = allnames[fixedpars==0]
labels = alllabels[fixedpars==0]
mytrue = pars_true_2beta[fixedpars==0]

mkrs = {}
for i in range(len(names)):
    mkrs[names[i]]=mytrue[i]

samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, samps_qp], filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=12
g.triangle_plot([samps, samps_qp], ['CMB', 'DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'], 
                filled=True,title_limit=2, legend_labels=['CMB-S4', 'S4-BI'], markers=mkrs)
suptitle('Marginalized over Synchrotron')
tight_layout()

averages_s4 = np.mean(flat_samples_s4, axis=0)
errors_s4 = np.std(flat_samples_s4, axis=0)
averages_qp = np.mean(flat_samples_qp, axis=0)
errors_qp = np.std(flat_samples_qp, axis=0)
print()
for i in range(len(names)):
    print('{0:20s}: CMBS4: {1:7.3f} +/- {2:7.3f}      -     S4BI: {3:7.3f} +/- {4:7.3f}'.format(names[i], averages_s4[i], errors_s4[i], averages_qp[i], errors_qp[i]))
print()
    
    
    
    
### Conversion of err_CMB into 
thecmb = where(names == 'CMB')[0][0]
Delta_r_CMB_s4 = averages_s4[thecmb]**2/0.345
Delta_r_CMB_qp = averages_qp[thecmb]**2/0.345
print('Delta r CMB S4 = {0:6.5g}'.format(Delta_r_CMB_s4))
print('Delta r CMB QP = {0:6.5g}'.format(Delta_r_CMB_qp))
print()

### Chi2
mypars_s4 = np.zeros(7)
mypars_s4[fixedpars==1] = p0[fixedpars==1]
mypars_s4[fixedpars==0] = averages_s4
mypars_qp = np.zeros(7)
mypars_qp[fixedpars==1] = p0[fixedpars==1]
mypars_qp[fixedpars==0] = averages_qp
# print(mypars_s4)
# print(mypars_qp)

chi2_s4 = np.sum(((sed_s4 - fg_S4_2b(nus_s4, mypars_s4))/err_sed_s4)**2)
ndf_s4 = len(sed_s4)-len(averages_s4)
chi2_qp = np.sum(((sed_qp - fg_qp_2b(nus_qp, mypars_qp))/err_sed_qp)**2)
ndf_qp = len(sed_qp)-len(averages_qp)
print('CMBS4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_s4, ndf_s4, chi2_s4/ndf_s4))
print('BI-S4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_qp, ndf_qp, chi2_qp/ndf_qp))


# Old Stuff

# Now we assume that a fraction of CMB-S4 is converted to BI

In [ ]:
reload(s4bi)
reload(mcmc)

f = 0.1

#### S4 Data
nus_s4 = s4bi.s4_config['frequency']
sed_s4 = model_2beta(nus_s4, pars_true)
err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(1-f)

#### S4BI Data
## Here we convert every band above 85 GHz to BI
qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])

## Here we convert only the two highest bands (220 and 270 GHz)
# qp_nsub = np.array([1, 1, 1, 1, 1, 1, 1, 1, 5, 5])
# qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
# qp_suboptimality = np.array([False, False, False, False, False, False, False, True, True])

qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
nus_qp = qp_config['frequency']
sed_qp = model_2beta(nus_qp, pars_true)
err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(f)

### Mix with a fraction f for BI
nus_both = np.append(nus_qp, nus_s4)
sed_both = np.append(sed_qp, sed_s4)
err_sed_both = np.append(err_sed_qp, err_sed_s4)



rc('figure',figsize=(8,8))
rc('font',size=12)

plot(nus, vals, label='All components')
plot(nus,pars_true[0]*sync_model.eval(nus, pars_true[1]), '--', label='Synchrotron ({})'.format(pars_true[1]))
#plot(nus,pars_true[2]+nus*0, '--', label='CMB ({})'.format(pars_true[2]))
p=plot(nus,pars_true[3]*double_dust_model.eval(nus, pars_true[4], pars_true[5], pars_true[6]) , '--', 
       label=r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
axvline(x=pars_true[6],ls=':', color=p[0].get_color(), label='Dust Break Frequency')
# errorbar(nus_qp, sed_qp, yerr=err_sed_qp, fmt='bo', capsize=4, label='S4-BI (f={})'.format(f))
# errorbar(nus_s4, sed_s4, yerr=err_sed_s4, fmt='ro', capsize=4, label='CMB-S4 (f={})'.format(f))
errorbar(nus_both, sed_both, yerr=err_sed_both, fmt='mo', 
         capsize=4, label='Combination {0:2.0f}% BI and {1:2.0f}% Imager'.format(100*f, 100*(1-f)))
yscale('log')
xscale('log')
ylim(1e-1, 1e2)
xlabel('Frequency [GHz]')
ylabel(r'SED [$\mu K_{CMB}$]')
title('fsky used: {0:3.1f}%'.format(myfsky*100))
legend()
savefig('SED_f_{0:2.0f}.png'.format(100*f))
show()

from qubic import mcmc
reload(mcmc)
p0 = pars_true.copy()

fixedpars = np.array([0, 0, 1, 0, 0, 0, 0])

ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
        p0=p0, model=model_2beta, fixedpars=fixedpars)
nbmc = 3000
samp = ll.run(nbmc)
flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)


ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
        p0=p0, model=model_2beta, fixedpars=fixedpars)
nbmc = 3000
samp_qp = ll_qp.run(nbmc)
flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)

ll_both = mcmc.LogLikelihood(xvals=nus_both, yvals=sed_both, errors=err_sed_both,
        p0=p0, model=model_2beta, fixedpars=fixedpars)
nbmc = 3000
samp_both = ll_both.run(nbmc)
flat_samples_both = samp_both.get_chain(discard=nbmc//2, thin=32, flat=True)

allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'])
alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta^0_{dust}', r'\beta^1_{dust}', r'\nu^{break}_{dust}'])

names = allnames[fixedpars==0]
labels = alllabels[fixedpars==0]
mytrue = pars_true[fixedpars==0]

mkrs = {}
for i in range(len(names)):
    mkrs[names[i]]=mytrue[i]

samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)
samps_both = MCSamples(samples=flat_samples_both, names=names, labels=labels)

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=10
g.triangle_plot([samps, samps_qp, samps_both], 
                filled=True,title_limit=None, 
                legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
                markers=mkrs,
               param_limits={'SyncAmp':[0.28, 0.3], 'SyncBeta':[-3.02,-2.98], 'DustAmp':[14,18],
                            'DustBeta0':[1.62,1.66], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#                            'DustBeta0':[1.3,1.5], 'DustBeta1':[1.2,2.2], 'DustNuBreak':[210, 270]})
savefig('BigTriangle_f_{0:2.0f}.png'.format(100*f))
show()

figure()
g = plots.getSubplotPlotter()
g.settings.scaling=False
g.settings.axes_fontsize=10
g.triangle_plot([samps, samps_qp, samps_both], ['DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'], 
                filled=True,title_limit=None, 
                legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
                markers=mkrs,
               param_limits={'DustAmp':[14,18],
                            'DustBeta0':[1.62,1.66], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#                            'DustBeta0':[1.3,1.5], 'DustBeta1':[1.2,2.2], 'DustNuBreak':[210, 270]})
savefig('TriangleMarginalized_f_{0:2.0f}.png'.format(100*f))
show()


averages = np.mean(flat_samples_both, axis=0)
errors = np.std(flat_samples_both, axis=0)
print(averages)
print(errors)
print()
for i in range(len(names)):
    print('{0:20s}: CMBS4: {1:7.3f} +/- {2:7.3f}      -     S4BI: {3:7.3f} +/- {4:7.3f}'.format(names[i], averages[i], errors[i], averages[i], errors[i]))



In [ ]:
# #'SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'

# figure()
# g = plots.getSubplotPlotter()
# g.settings.scaling=False
# g.settings.axes_fontsize=10
# g.triangle_plot([samps, samps_qp, samps_both], 
#                 filled=True,title_limit=None, 
#                 legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                 markers=mkrs,
#                param_limits={'SyncAmp':[0.28, 0.3], 'SyncBeta':[-3.02,-2.98], 'DustAmp':[14,18],
#                             'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})

# figure()
# g = plots.getSubplotPlotter()
# g.settings.scaling=False
# g.settings.axes_fontsize=10
# g.triangle_plot([samps, samps_qp, samps_both], ['DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'], 
#                 filled=True,title_limit=None, 
#                 legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                 markers=mkrs,
#                param_limits={'DustAmp':[14,18],
#                             'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
# #suptitle('Marginalized over synchrotron')


# Loop over values of f to make an animation

In [ ]:
reload(s4bi)
reload(mcmc)

allf = np.array([0.999999, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.00001])
all_averages = np.zeros((len(allf), 6))
all_errors = np.zeros((len(allf), 6))


for j in range(len(allf)):
    f = allf[j]
    #### S4 Data
    nus_s4 = s4bi.s4_config['frequency']
    sed_s4 = model_2beta(nus_s4, pars_true)
    err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(1-f)

    #### S4BI Data
    ## Here we convert every band above 85 GHz to BI
#     qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
#     qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
#     qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])

    ## Here we convert only the two highest bands (220 and 270 GHz)
    qp_nsub = np.array([1, 1, 1, 1, 1, 1, 1, 1, 5, 5])
    qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
    qp_suboptimality = np.array([False, False, False, False, False, False, False, True, True])

    qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
    nus_qp = qp_config['frequency']
    sed_qp = model_2beta(nus_qp, pars_true)
    err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(f)

    ### Mix with a fraction f for BI
    nus_both = np.append(nus_qp, nus_s4)
    sed_both = np.append(sed_qp, sed_s4)
    err_sed_both = np.append(err_sed_qp, err_sed_s4)



    rc('figure',figsize=(8,8))
    rc('font',size=12)

#     plot(nus, vals, label='All components')
#     plot(nus,pars_true[0]*sync_model.eval(nus, pars_true[1]), '--', label='Synchrotron ({})'.format(pars_true[1]))
#     #plot(nus,pars_true[2]+nus*0, '--', label='CMB ({})'.format(pars_true[2]))
#     p=plot(nus,pars_true[3]*double_dust_model.eval(nus, pars_true[4], pars_true[5], pars_true[6]) , '--', 
#            label=r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
#     axvline(x=pars_true[6],ls=':', color=p[0].get_color(), label='Dust Break Frequency')
#     # errorbar(nus_qp, sed_qp, yerr=err_sed_qp, fmt='bo', capsize=4, label='S4-BI (f={})'.format(f))
#     # errorbar(nus_s4, sed_s4, yerr=err_sed_s4, fmt='ro', capsize=4, label='CMB-S4 (f={})'.format(f))
#     errorbar(nus_both, sed_both, yerr=err_sed_both, fmt='mo', 
#              capsize=4, label='Combination {0:2.0f}% BI and {1:2.0f}% Imager'.format(100*f, 100*(1-f)))
#     yscale('log')
#     xscale('log')
#     ylim(1e-1, 1e2)
#     xlabel('Frequency [GHz]')
#     ylabel(r'SED [$\mu K_{CMB}$]')
#     title('fsky used: {0:3.1f}%'.format(myfsky*100))
#     legend()
#     savefig('SED_f_{0:2.0f}.png'.format(100*f))
#     show()

    from qubic import mcmc
    reload(mcmc)
    p0 = pars_true.copy()

    fixedpars = np.array([0, 0, 1, 0, 0, 0, 0])

#     ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
#             p0=p0, model=model_2beta, fixedpars=fixedpars)
#     nbmc = 3000
#     samp = ll.run(nbmc)
#     flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)


#     ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
#             p0=p0, model=model_2beta, fixedpars=fixedpars)
#     nbmc = 3000
#     samp_qp = ll_qp.run(nbmc)
#     flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)

    ll_both = mcmc.LogLikelihood(xvals=nus_both, yvals=sed_both, errors=err_sed_both,
            p0=p0, model=model_2beta, fixedpars=fixedpars)
    nbmc = 3000
    samp_both = ll_both.run(nbmc)
    flat_samples_both = samp_both.get_chain(discard=nbmc//2, thin=32, flat=True)

    allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'])
    alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta^0_{dust}', r'\beta^1_{dust}', r'\nu^{break}_{dust}'])

    names = allnames[fixedpars==0]
    labels = alllabels[fixedpars==0]
    mytrue = pars_true[fixedpars==0]

    mkrs = {}
    for i in range(len(names)):
        mkrs[names[i]]=mytrue[i]

#     samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
#     samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)
    samps_both = MCSamples(samples=flat_samples_both, names=names, labels=labels)

#     figure()
#     g = plots.getSubplotPlotter()
#     g.settings.scaling=False
#     g.settings.axes_fontsize=10
#     g.triangle_plot([samps, samps_qp, samps_both], 
#                     filled=True,title_limit=None, 
#                     legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                     markers=mkrs,
#                    param_limits={'SyncAmp':[0.28, 0.3], 'SyncBeta':[-3.02,-2.98], 'DustAmp':[14,18],
#                                 'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#     savefig('BigTriangle_f_{0:2.0f}.png'.format(100*f))
#     show()

#     figure()
#     g = plots.getSubplotPlotter()
#     g.settings.scaling=False
#     g.settings.axes_fontsize=10
#     g.triangle_plot([samps, samps_qp, samps_both], ['DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'], 
#                     filled=True,title_limit=None, 
#                     legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                     markers=mkrs,
#                    param_limits={'DustAmp':[14,18],
#                                 'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#     savefig('TriangleMarginalized_f_{0:2.0f}.png'.format(100*f))
#     show()

#     figure()
#     g = plots.getSubplotPlotter()
#     g.settings.scaling=False
#     g.settings.axes_fontsize=10
#     g.triangle_plot([samps_both], 
#                     filled=True,title_limit=None, 
#                     legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                     markers=mkrs,
#                    param_limits={'SyncAmp':[0.28, 0.3], 'SyncBeta':[-3.02,-2.98], 'DustAmp':[14,18],
#                                 'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#     savefig('BigTriangle_f_{0:2.0f}.png'.format(100*f))
#     show()

#     figure()
#     g = plots.getSubplotPlotter()
#     g.settings.scaling=False
#     g.settings.axes_fontsize=10
#     g.triangle_plot([samps_both], ['DustAmp', 'DustBeta0', 'DustBeta1', 'DustNuBreak'], 
#                     filled=True,title_limit=None, 
#                     legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'], 
#                     markers=mkrs,
#                    param_limits={'DustAmp':[14,18],
#                                 'DustBeta0':[1.65,1.73], 'DustBeta1':[1.,1.55], 'DustNuBreak':[210, 270]})
#     savefig('TriangleMarginalized_f_{0:2.0f}.png'.format(100*f))
#     show()

    averages = np.mean(flat_samples_both, axis=0)
    errors = np.std(flat_samples_both, axis=0)
    print(averages)
    print(errors)
    
    all_averages[j,:] = averages
    all_errors[j,:] = errors


In [ ]:
all_errors

In [ ]:
### Result of the above when converting all bands above 85 GHz to BI
all_errors_allabove85 = array([[6.69205855e-04, 1.98324488e-03, 4.97252011e-02, 3.24193421e-03,
        1.20407296e-02, 2.76652685e+00],
       [6.81653818e-04, 2.01866536e-03, 5.26572216e-02, 3.16232773e-03,
        1.24194899e-02, 2.62885091e+00],
       [6.93358607e-04, 2.03617210e-03, 5.71630483e-02, 2.99789272e-03,
        1.37592893e-02, 2.76992777e+00],
       [6.54058098e-04, 1.93072058e-03, 5.85983298e-02, 2.92392115e-03,
        1.38473229e-02, 2.77418569e+00],
       [7.08292784e-04, 2.07986431e-03, 6.37278770e-02, 2.90004976e-03,
        1.51155748e-02, 2.92976843e+00],
       [6.75181105e-04, 1.97834579e-03, 7.04440542e-02, 2.87132850e-03,
        1.65744377e-02, 3.01944849e+00],
       [6.68190826e-04, 1.96812353e-03, 7.98012698e-02, 2.76531559e-03,
        1.85421565e-02, 3.25986315e+00],
       [6.49721524e-04, 1.91089230e-03, 9.49505678e-02, 2.68850392e-03,
        2.20432580e-02, 3.66340011e+00],
       [6.70549439e-04, 1.98993387e-03, 1.08356705e-01, 2.62925626e-03,
        2.49549731e-02, 4.08196652e+00],
       [6.67774459e-04, 1.97011999e-03, 1.47691409e-01, 2.64158211e-03,
        3.37456769e-02, 5.43217092e+00],
       [6.64149050e-04, 1.93866955e-03, 4.84303920e-01, 2.56579357e-03,
        1.14083352e-01, 9.66406295e+00]])

### Now when we only convert the 220 and 270 bands
all_errors_220_270 = array([[6.60024002e-04, 1.95051299e-03, 5.62226922e-02, 2.84239736e-03,
        1.38294838e-02, 2.99767744e+00],
       [6.93821076e-04, 2.04226326e-03, 5.92834136e-02, 2.87519808e-03,
        1.47012014e-02, 3.16569081e+00],
       [6.64617121e-04, 1.95469423e-03, 6.12953203e-02, 2.72088269e-03,
        1.50357296e-02, 3.16531919e+00],
       [6.71593954e-04, 1.98678454e-03, 6.65716926e-02, 2.66934256e-03,
        1.60840630e-02, 3.20815514e+00],
       [6.79091055e-04, 2.01486418e-03, 7.13322115e-02, 2.69668157e-03,
        1.70041820e-02, 3.18635773e+00],
       [6.85526565e-04, 2.02682914e-03, 7.61477595e-02, 2.67678600e-03,
        1.81296446e-02, 3.36336971e+00],
       [6.40424061e-04, 1.90247262e-03, 8.60077394e-02, 2.67228150e-03,
        2.03592212e-02, 3.67210606e+00],
       [6.66384738e-04, 1.95752873e-03, 9.58608787e-02, 2.66692821e-03,
        2.25567227e-02, 4.05476333e+00],
       [6.87166039e-04, 2.03688255e-03, 1.23924060e-01, 2.60793677e-03,
        2.87978971e-02, 4.89799089e+00],
       [6.51144728e-04, 1.90086422e-03, 1.62377267e-01, 2.64329249e-03,
        3.73585610e-02, 6.29614051e+00],
       [6.61469066e-04, 1.96064897e-03, 4.99555569e-01, 2.59432199e-03,
        1.17338355e-01, 1.05893760e+01]])

# What fraction in BI would allow to reach CMB-S4 science target ?
Conversion of dust amplitude at 353 GHz to a $\Delta(r)$ is made the following way:
- we convert to 150 GHz (ratio is 0.047 with standard modified black-body and $\beta=1.54$) and get a RMS in $\mu K$.
- the we convert this into a $\Delta r$ using a template of pure primordial BB map RMS as a function of r. See Notebook "RMS_Map_r.Rmd)

In [ ]:
rc('figure',figsize=(8,6))
rc('font',size=12)
all_errors = all_errors_allabove85.copy()
figure()
plot(allf*100, all_errors[:,2],'ro-', label=r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
xlabel('BI Fraction [%]')
ylabel(r'Dust Amplitude Error at 353 GHz [$\mu K$]')
yscale('log')
title('Combination Imager/BI')
legend()



figure()
plot(allf*100, (all_errors[:,2]*0.047)**2/0.345,'ro-', label=r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
axhline(y=5e-4,ls='--',color='b',label='CMB-S4 Science Target [Abazajian et al., ArXiv:2008.12619]')
xlabel('BI Fraction [%]')
ylabel(r'$\Delta(r)$')
ylim(1e-5, 1e-2)
xlim(0,100)
yscale('log')
legend()


In [ ]:
rc('figure',figsize=(10,6))
rc('font',size=12)
figure()
plot(allf*100, all_errors_allabove85[:,2],'ro-', label='All bands Bands above 85 GHz (included)')
plot(allf*100, all_errors_220_270[:,2],'bo-', label='Only 220 and 270 Bands')
xlabel('BI Fraction [%]')
ylabel(r'Dust Amplitude Error at 353 GHz [$\mu K$]')
yscale('log')
title('Combination Imager/BI \n'+r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
legend()



figure()
plot(allf*100, (all_errors_allabove85[:,2]*0.047)**2/0.345,'ro-', label='All bands Bands above 85 GHz (included)')
plot(allf*100, (all_errors_220_270[:,2]*0.047)**2/0.345,'bo-', label='Only 220 and 270 GHz Bands')
axhline(y=5e-4,ls='--',color='g',label='CMB-S4 Science Target [Abazajian et al., ArXiv:2008.12619]')
xlabel('BI Fraction [%]')
ylabel(r'$\Delta(r)$ (bias on r from dust residuals)')
ylim(1e-5, 1e-2)
xlim(0,100)
yscale('log')
title('Combination Imager/BI \n'+r'Double Beta Dust ($\beta_0$={0:5.2f},$\beta_1=${1:5.2f},$\nu_0=${2:3.0f} GHz)'.format(pars_true[4], pars_true[5], pars_true[6]))
legend()


# Fit of the 2β with a 1β model and chi2 comparison

In [ ]:
f = 0.1

#### S4 Data
nus_s4 = s4bi.s4_config['frequency']
sed_s4 = model_2beta(nus_s4, pars_true)
err_sed_s4 = s4bi.s4_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(1-f)

#### S4BI Data
qp_nsub = np.array([1, 1, 1, 5, 5, 5, 5, 5, 5, 5])
qp_effective_fraction = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1.])
qp_suboptimality = np.array([False, False, False, True, True, True, True, True, True])
qp_config = s4bi.qubicify(s4bi.s4_config, qp_nsub, qp_effective_fraction, suboptimality=qp_suboptimality)
nus_qp = qp_config['frequency']
sed_qp = model_2beta(nus_qp, pars_true)
err_sed_qp = qp_config['depth_p'] / np.sqrt(arcmins_squared) / np.sqrt(f)

### Mix with a fraction f for BI
nus_both = np.append(nus_qp, nus_s4)
sed_both = np.append(sed_qp, sed_s4)
err_sed_both = np.append(err_sed_qp, err_sed_s4)



### Fitting with the 1beta model
p0 = np.array([s_sync_70GHz[3], -3, 0, s_dust_353GHz[3], 1.54])
fixedpars = np.array([0, 0, 0, 0, 0])


ll = mcmc.LogLikelihood(xvals=nus_s4, yvals=sed_s4, errors=err_sed_s4,
        p0=p0, model=model_1beta, fixedpars=fixedpars)
nbmc = 3000
samp = ll.run(nbmc)
flat_samples_s4 = samp.get_chain(discard=nbmc//2, thin=32, flat=True)

ll_qp = mcmc.LogLikelihood(xvals=nus_qp, yvals=sed_qp, errors=err_sed_qp,
        p0=p0, model=model_1beta, fixedpars=fixedpars)
nbmc = 3000
samp_qp = ll_qp.run(nbmc)
flat_samples_qp = samp_qp.get_chain(discard=nbmc//2, thin=32, flat=True)

ll_both = mcmc.LogLikelihood(xvals=nus_both, yvals=sed_both, errors=err_sed_both,
        p0=p0, model=model_1beta, fixedpars=fixedpars)
nbmc = 3000
samp_both = ll_both.run(nbmc)
flat_samples_both = samp_both.get_chain(discard=nbmc//2, thin=32, flat=True)


In [ ]:
allnames = np.array(['SyncAmp', 'SyncBeta', 'CMB', 'DustAmp', 'DustBeta'])
alllabels = np.array(['A_{sync}', r'\beta_{sync}', 'A_{CMB}', 'A_{dust}', r'\beta_{dust}'])
names = allnames[fixedpars==0]
labels = alllabels[fixedpars==0]


# samps = MCSamples(samples=flat_samples_s4, names=names, labels=labels)
# samps_qp = MCSamples(samples=flat_samples_qp, names=names, labels=labels)
# samps_both = MCSamples(samples=flat_samples_both, names=names, labels=labels)

# figure()
# g = plots.getSubplotPlotter()
# g.settings.scaling=False
# g.settings.axes_fontsize=10
# g.triangle_plot([samps, samps_qp, samps_both], 
#                 filled=True,title_limit=None, 
#                 legend_labels=['CMB-S4 {0:2.0f}%'.format(100*(1-f)), 'S4-BI {0:2.0f}%'.format(100*f), 'Both'])

averages_s4 = np.mean(flat_samples_s4, axis=0)
errors_s4 = np.std(flat_samples_s4, axis=0)

averages_qp = np.mean(flat_samples_qp, axis=0)
errors_qp = np.std(flat_samples_qp, axis=0)

averages = np.mean(flat_samples_both, axis=0)
errors = np.std(flat_samples_both, axis=0)


print()
print('CMB-S4')
for i in range(len(names)):
    print('{0:20s}: {1:7.3f} +/- {2:7.3f} '.format(names[i], averages_s4[i], errors_s4[i]))

print()
print('BI-S4')
for i in range(len(names)):
    print('{0:20s}: {1:7.3f} +/- {2:7.3f} '.format(names[i], averages_qp[i], errors_qp[i]))

print()
print('{0:3.0f}% Bolometric Interferometer'.format(f*100))
for i in range(len(names)):
    print('{0:20s}: {1:7.3f} +/- {2:7.3f} '.format(names[i], averages[i], errors[i]))



In [ ]:
Delta_r_CMB = 0.021**2/0.345
print('Delta r CMB = {0:5.4f}'.format(Delta_r_CMB))

In [ ]:
### Chi2
chi2_s4 = np.sum(((sed_s4 - model_1beta(nus_s4, averages_s4))/err_sed_s4)**2)
ndf_s4 = len(sed_s4)-len(averages_s4)
chi2_qp = np.sum(((sed_qp - model_1beta(nus_qp, averages_qp))/err_sed_qp)**2)
ndf_qp = len(sed_qp)-len(averages_qp)
chi2 = np.sum(((sed_both - model_1beta(nus_both, averages))/err_sed_both)**2)
ndf = len(sed_both)-len(averages)
print('CMBS4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_s4, ndf_s4, chi2_s4/ndf_s4))
print('BI-S4: chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2_qp, ndf_qp, chi2_qp/ndf_qp))
print('Both : chi2={0:5.1f} ndf={1:3.0f} chi2/ndf={2:5.2f}'.format(chi2, ndf, chi2/ndf))
